In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15161972
paper_name = 'askree_mceachern_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [41]:
original_data = pd.read_excel('raw_data/01263Table3.xlsx', sheet_name='Sheet1', skiprows=1)

In [42]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 183 x 5


In [43]:
original_data.head()

,ORF name,Common name,Measurement 1,Measurement 2,Comments
0,YLR233C,EST1,-293.3,NaN,NaN
1,YBL088C,TEL1,-240,NaN,NaN
2,YNL250W,RAD50,-233.2,NaN,NaN
3,YMR106C,YKU80,-227,NaN,NaN
4,YDR369C,XRS2,-225,NaN,NaN


In [44]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [45]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [46]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [47]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                      ORF name Common name  \
index_input                                                                  
181                                                        NaN         NaN   
182          Measured length differences of the Y' containi...         NaN   

            Measurement 1  Measurement 2 Comments  \
index_input                                         
181                   NaN            NaN      NaN   
182                   NaN            NaN      NaN   

                                                           orf  
index_input                                                     
181                                                        NAN  
182          MEASUREDLENGTHDIFFERENCESOFTHEYCONTAININGTELOM...  


In [48]:
original_data = original_data.loc[t,:]

In [49]:
original_data.set_index('orf', inplace=True)

In [50]:
original_data = original_data[['Measurement 1','Measurement 2']].apply(pd.to_numeric, axis=1, errors='coerce').mean(axis=1).to_frame()

In [51]:
original_data = original_data.groupby(original_data.index).mean()

In [52]:
original_data.shape

(181, 1)

# Load & process tested strains

In [53]:
tested = pd.read_excel('raw_data/S.cerftpmata.xlsx', sheet_name='Sheet1', skiprows=2)

In [54]:
tested.head()

,record no.,ORF name,strain,batch,row/col,re-array,re-array.1,re-array.2,RG notes,QC Notes,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,plate,row,column,NaN,problem,comments
1,338.0,YAL068C,BY4741,01_1,A2,1,A,2,NaN,NaN,NaN
2,339.0,YAL067C,BY4741,01_1,A3,1,A,3,NaN,NaN,NaN
3,340.0,YAL066W,BY4741,01_1,A4,1,A,4,NaN,NaN,NaN
4,341.0,YAL065C,BY4741,01_1,A5,1,A,5,NaN,NaN,NaN


In [55]:
tested['orf'] = tested['ORF name'].astype(str)

In [56]:
tested['orf'] = clean_orf(tested['orf'])

In [57]:
typo_fixes = {'YOLO57W':'YOL057W','YOLO62C':'YOL062C','YKLO72W':'YKL072W','YJL206-A':'YJL206C-A','YLR287-A':'YLR287C-A'}
tested['orf'] = tested['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [58]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [59]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name  strain batch row/col re-array re-array.1  \
index_input                                                                  
0                   NaN      NaN     NaN   NaN     NaN    plate        row   
1964             5242.0        .  BY4741  12_5     A10       21          H   
4711                NaN      NaN     NaN   NaN     NaN      NaN        NaN   

            re-array.2 RG notes QC Notes Unnamed: 10  orf  
index_input                                                
0               column      NaN  problem    comments  NAN  
1964                 1      NaN      NaN         NaN       
4711               NaN      NaN      NaN         NaN  NAN  


In [60]:
tested = tested.loc[t,:]

In [61]:
tested_orfs = tested['orf'].unique()

In [62]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [63]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [64]:
original_data.head()

,0
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


# Prepare the final dataset

In [65]:
data = original_data.copy()

In [66]:
dataset_ids = [113]
datasets = datasets.reindex(index=dataset_ids)

In [67]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [68]:
data.head()

dataset_id,113
data_type,value
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


## Subset to the genes currently in SGD

In [69]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [70]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,113
,data_type,value
gene_id,orf,
1869,YAL068C,0.0
61,YAL067C,0.0
60,YAL066W,0.0
1727,YAL065C,0.0
57,YAL062W,0.0


# Normalize

In [71]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [72]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [73]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        113          
data_type       value    valuez
gene_id orf                    
1869    YAL068C   0.0  0.113346
61      YAL067C   0.0  0.113346
60      YAL066W   0.0  0.113346
1727    YAL065C   0.0  0.113346
57      YAL062W   0.0  0.113346

# Print out

In [74]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [75]:
from IO.save_data_to_db3 import *

In [76]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15161972...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.32s/it]

Updating the data_modified_on field...
